# Check EMA Compliance

In [1]:
import pandas as pd
import datetime as dt
from datetime import date
import numpy as np

today = date.today() 
week_ago = today - dt.timedelta(days=7)
today = today.strftime("%d%m%Y")

week_ago = week_ago.strftime("%Y-%m-%d")

datapath = f"/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/export_tiki_{today}"
filepath = "/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/"

In [2]:
session = pd.read_csv(datapath + "/questionnaireSession.csv")

In [3]:
session["user"] = session["user"].str[:4]

In [4]:
session.head()

,id,user,study,questionnaire,sessionRun,expirationTimestamp,createdAt,completedAt
0,6954,APbN,25,56,0,1682583300000,1682582801477,1.682583e+12
1,6963,APbN,25,68,0,1682590500000,1682589815644,1.682590e+12
2,7004,APbN,25,76,0,1682684100000,1682683238257,1.682683e+12
3,7010,APbN,25,84,0,1682691300000,1682690465941,1.682691e+12
4,7012,APbN,25,90,0,1682694900000,1682694098187,1.682694e+12


In [5]:
answers = pd.read_csv(datapath + "/answers.csv")

In [6]:
answers["user"] = answers["user"].str[:4]

In [7]:
answers.head()

,id,user,questionnaireSession,study,questionnaire,question,order,createdAt,element,value
0,17109,APbN,6954,25,56,315,0,1682582834748,1707,NaN
1,17110,APbN,6954,25,56,316,0,1682582835748,1720,NaN
2,17111,APbN,6954,25,56,317,0,1682582836645,1726,NaN
3,17112,APbN,6954,25,56,318,0,1682582837559,1734,NaN
4,17113,APbN,6954,25,56,319,0,1682582838516,1740,NaN


In [8]:
answers = answers[["id", "user", "questionnaireSession", "questionnaire", "element"]]

In [9]:
choice = pd.read_csv(datapath + "/choice.csv")

In [10]:
choice = choice[["element", "choice_id", "text", "question"]]

In [11]:
choice.head()

,element,choice_id,text,question
0,1707,1,1,315
1,1708,2,2,315
2,1709,3,3,315
3,1710,4,4,315
4,1711,5,5,315


### Combine with EMA ID 

In [12]:
#df_ans = answers_merged[["user", "questionnaireSession", "choice_id"]]
#df_ans.rename(columns = {"user":"customer"}, inplace=True)

In [13]:
df_sess = session[["user", "questionnaire", "sessionRun", "createdAt", "completedAt"]]


In [14]:
df_sess.rename(columns = {"user":"customer"}, inplace=True)

/Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [15]:
sess_count1 = df_sess.dropna(subset=["completedAt"]).groupby("customer")["completedAt"].size()

In [16]:
df_active = pd.read_csv(filepath +"/active_participants.csv")

In [17]:
df_active.rename(columns = {"Pseudonym ": "customer", "EMA ID": "ema_id", "Ende EMA Baseline": "end_ema", "Status": "status",
                            "Start EMA Baseline": "start_ema", "Studienversion":"study_version", "FOR_ID":"for_id"}, inplace=True)

df_active = df_active[["customer", "ema_id", "end_ema", "start_ema", "study_version", "for_id", "status"]]

In [18]:
df_active["customer"] = df_active.customer.str[:4]

In [19]:
df_complete = pd.merge(df_active, sess_count1, on="customer")

In [20]:
df_complete.tail()

,customer,ema_id,end_ema,start_ema,study_version,for_id,status,completedAt
62,yv0Q,EMA3016,02.11.2023,19.09.2023,Lang,FOR13012,Erhebung_1_aktiv,18
63,5Ger,EMA1020,02.11.2023,19.09.2023,Kurz,FOR11036,Erhebung_1_aktiv,41
64,7uRT,EMA4030,05.10.2023,21.09.2023,Lang,FOR14024,Erhebung_1_aktiv,6
65,5BAw,EMA4032,06.10.2023,22.09.2023,Lang,FOR14025,Erhebung_1_aktiv,22
66,Snml,EMA4034,09.10.2023,25.09.2023,Lang,FOR14026,Erhebung_1_aktiv,5


In [21]:
df_finish = df_complete[df_complete.status == "Abgeschlossen"]

In [22]:
df_finish

,customer,ema_id,end_ema,start_ema,study_version,for_id,status,completedAt
4,5qL5,EMA4001,26.06.2023,12.06.2023,Kurz,FOR14903,Abgeschlossen,33
5,L8UG,EMA1006,29.06.2023,15.06.2023,Kurz,FOR11010,Abgeschlossen,24
8,3ZqH,EMA3002,04.07.2023,20.06.2023,Kurz,FOR13901\n,Abgeschlossen,67
14,3s90,EMA3004,11.07.2023,27.06.2023,Kurz,FOR13001,Abgeschlossen,21
16,f7RF,EMA4005,13.07.2023,29.06.2023,Kurz,FOR14006,Abgeschlossen,76
21,bVec,EMA1007,20.07.2023,06.07.2023,Kurz,FOR11018,Abgeschlossen,91
26,Emob,EMA1011,25.07.2023,11.07.2023,Kurz,FOR11020,Abgeschlossen,49
28,uN1K,EMA4009,26.07.2023,12.07.2023,Kurz (Wechsel/Abbruch),FOR14011,Abgeschlossen,23
30,bNPw,EMA1013,02.08.2023,19.07.2023,Kurz,FOR11023,Abgeschlossen,25
34,GuiZ,EMA2005,08.08.2023,25.07.2023,Kurz (Wechsel/Abbruch),FOR12002,Abgeschlossen,25


In [23]:
df_finish.completedAt.describe()

count    15.000000
mean     49.466667
std      28.525344
min      21.000000
25%      24.500000
50%      37.000000
75%      71.500000
max      98.000000
Name: completedAt, dtype: float64

In [24]:
df_finish_long = df_complete[df_complete.status == "Post_Erhebung_1"]

In [25]:
df_finish_long.completedAt.describe()

count     38.000000
mean      67.157895
std       32.563058
min        1.000000
25%       39.250000
50%       70.000000
75%       96.500000
max      113.000000
Name: completedAt, dtype: float64

In [26]:
df_finish_long

,customer,ema_id,end_ema,start_ema,study_version,for_id,status,completedAt
0,4MLe,EMA1001,31.05.2023,17.05.2023,Lang,FOR11905,Post_Erhebung_1,97
1,kVhY,EMA1003,09.06.2023,26.05.2023,Lang,FOR11001,Post_Erhebung_1,92
2,N3CY,EMA1002,14.06.2023,31.05.2023,Lang,FOR11003,Post_Erhebung_1,19
3,8ear,EMA1004,20.06.2023,06.06.2023,Lang,FOR11005,Post_Erhebung_1,49
6,94FE,EMA2002,29.06.2023,15.06.2023,Lang,FOR12901,Post_Erhebung_1,103
7,asYV,EMA1008,04.07.2023,20.06.2023,Lang,FOR11011,Post_Erhebung_1,15
9,wGQ8,EMA1005,05.07.2023,21.06.2023,Lang (Wechsel),FOR11012,Post_Erhebung_1,17
10,gfo4,EMA4003,06.07.2023,22.06.2023,Lang (Wechsel),FOR14002,Post_Erhebung_1,28
11,kH1x,EMA4002,06.07.2023,22.06.2023,Lang,FOR14003,Post_Erhebung_1,52
12,MJFT,EMA4004,07.07.2023,23.06.2023,Lang,FOR14005,Post_Erhebung_1,62
